In [1]:
%load_ext autoreload

%autoreload 2
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, roc_curve
from ast import literal_eval

import os
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegressionCV, LinearRegression

import warnings
import numpy as np
from collections import OrderedDict

from lob_data_utils import lob, db_result, overview, roc_results, model
from lob_data_utils.svm_calculation import lob_svm

sns.set_style('whitegrid')
warnings.filterwarnings('ignore')

/home/vevurka/.conda/envs/py36/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
data_length = 10000 # TODO: not used
stock = '9064'

In [3]:
df, df_test = lob.load_prepared_data(
    stock, data_dir='../queue_imbalance/data/prepared', cv=False, include_test=True, length=None)
print(len(df), len(df_test))

14141 3535


In [4]:
def prepare_df(df):
    df.rename(columns={'Unnamed: 0': 'datetime'}, inplace=True)
    df.index = df['datetime']
    df = df.sort_index()
    df['prev_queue_imbalance'] = [None] + df['queue_imbalance'].iloc[0:len(df)-1].values.tolist()
    df.dropna(inplace=True)
    return df
df = prepare_df(df)
df_test = prepare_df(df_test)
df.head()

,datetime,bid,ask,bid_price,ask_price,mid_price,sum_sell_ask,sum_buy_bid,mid_price_indicator,queue_imbalance,prev_queue_imbalance
datetime,,,,,,,,,,,
2013-09-16 12:24:00,2013-09-16 12:24:00,"[(35.0, 50000.0), (40.0, 12000.0), (42.0, 5000...","[(77.1, 53636.0), (77.11, 7592.0), (77.12, 219...",77.08,77.10,77.090,53636.0,100.0,1.0,-0.996278,0.261169
2013-09-16 12:25:00,2013-09-16 12:25:00,"[(35.0, 50000.0), (40.0, 12000.0), (42.0, 5000...","[(77.12, 26303.0), (77.13, 25876.0), (77.14, 3...",77.09,77.12,77.105,26303.0,31203.0,1.0,0.085208,-0.996278
2013-09-16 12:26:00,2013-09-16 12:26:00,"[(35.0, 50000.0), (40.0, 12000.0), (42.0, 5000...","[(77.16, 3400.0), (77.17, 23792.0), (77.18, 10...",77.13,77.16,77.145,3400.0,25554.0,1.0,0.765145,0.085208
2013-09-16 12:27:00,2013-09-16 12:27:00,"[(35.0, 50000.0), (40.0, 12000.0), (42.0, 5000...","[(77.19, 6735.0), (77.2, 39762.0), (77.21, 542...",77.16,77.19,77.175,6735.0,30903.0,1.0,0.642117,0.765145
2013-09-16 12:28:00,2013-09-16 12:28:00,"[(35.0, 50000.0), (40.0, 12000.0), (42.0, 5000...","[(77.19, 4610.0), (77.2, 56499.0), (77.21, 377...",77.17,77.19,77.180,4610.0,17290.0,1.0,0.578995,0.642117


In [5]:
def get_bid_price(df: pd.DataFrame, index: int) -> float:
    bid_list = literal_eval(df['bid'].iloc[index])
    if not np.any(bid_list):
        return 0
    return np.max([price for price, vol in bid_list])


def get_ask_price(df: pd.DataFrame, index: int) -> float:
    ask_list = literal_eval(df['ask'].iloc[index])
    if not np.any(ask_list):
        return 0
    return np.min([price for price, vol in ask_list])

def sum_buy_active_orders(price: float, df: pd.DataFrame, index: int) -> float:
    bid = literal_eval(df['bid'].iloc[index])
    if not np.any(bid):
        return 0
    print('len, ', [(p, vol) for p, vol in bid if p == price])
    return sum([vol for p, vol in bid if p == price])


def sum_sell_active_orders(price: float, df: pd.DataFrame, index: int) -> float:
    ask = literal_eval(df['ask'].iloc[index])
    if not np.any(ask):
        return 0
    print('len, ', [(p, vol) for p, vol in ask if p == price])
    return sum([vol for p, vol in ask if p == price])


def queue_imbalance_(df: pd.DataFrame, index: int):
    sum_buy = sum_buy_active_orders(get_bid_price(df, index), df, index)
    sum_ask = sum_sell_active_orders(get_ask_price(df, index), df, index)
    diff_bid_ask = sum_buy - sum_ask
    sum_bid_ask = sum_buy + sum_ask
    if sum_bid_ask == 0:
        return 0  # as the lists are the same length 0
    return diff_bid_ask / sum_bid_ask

In [6]:
def calculate_queue_imbalance2(df: pd.DataFrame, index: int):
    bid_list = np.array(literal_eval(df['bid'].iloc[index]))
    ask_list = np.array(literal_eval(df['ask'].iloc[index]))
    bid_price, bid_vol = bid_list[::-1][1]
    ask_price, ask_vol = ask_list[1]
    
    diff_bid_ask = bid_vol - ask_vol
    sum_bid_ask = bid_vol + ask_vol
    if sum_bid_ask == 0:
        return 0  # as the lists are the same length 0
    return diff_bid_ask / sum_bid_ask

In [7]:
calculate_queue_imbalance2(df, 0), df['queue_imbalance'].iloc[0]

(0.05548643941278925, -0.9962781003424148)

In [8]:
q = []
for i in range(len(df)):
    q.append(calculate_queue_imbalance2(df, i))
df['queue_imbalance_2'] = q
df.head()


,datetime,bid,ask,bid_price,ask_price,mid_price,sum_sell_ask,sum_buy_bid,mid_price_indicator,queue_imbalance,prev_queue_imbalance,queue_imbalance_2
datetime,,,,,,,,,,,,
2013-09-16 12:24:00,2013-09-16 12:24:00,"[(35.0, 50000.0), (40.0, 12000.0), (42.0, 5000...","[(77.1, 53636.0), (77.11, 7592.0), (77.12, 219...",77.08,77.10,77.090,53636.0,100.0,1.0,-0.996278,0.261169,0.055486
2013-09-16 12:25:00,2013-09-16 12:25:00,"[(35.0, 50000.0), (40.0, 12000.0), (42.0, 5000...","[(77.12, 26303.0), (77.13, 25876.0), (77.14, 3...",77.09,77.12,77.105,26303.0,31203.0,1.0,0.085208,-0.996278,0.188573
2013-09-16 12:26:00,2013-09-16 12:26:00,"[(35.0, 50000.0), (40.0, 12000.0), (42.0, 5000...","[(77.16, 3400.0), (77.17, 23792.0), (77.18, 10...",77.13,77.16,77.145,3400.0,25554.0,1.0,0.765145,0.085208,-0.336179
2013-09-16 12:27:00,2013-09-16 12:27:00,"[(35.0, 50000.0), (40.0, 12000.0), (42.0, 5000...","[(77.19, 6735.0), (77.2, 39762.0), (77.21, 542...",77.16,77.19,77.175,6735.0,30903.0,1.0,0.642117,0.765145,-0.557004
2013-09-16 12:28:00,2013-09-16 12:28:00,"[(35.0, 50000.0), (40.0, 12000.0), (42.0, 5000...","[(77.19, 4610.0), (77.2, 56499.0), (77.21, 377...",77.17,77.19,77.180,4610.0,17290.0,1.0,0.578995,0.642117,-0.487227


## Let's try to fit Logistic Regression

In [9]:
df_result = pd.DataFrame()
features = ['queue_imbalance', 'queue_imbalance_2']
res = model.validate_model(LogisticRegressionCV(), 
                           df[features], df['mid_price_indicator'])
res['features'] = ', '.join(features)
res['method'] = 'logistic'
df_result = df_result.append(res, ignore_index=True)

features = ['queue_imbalance']
res = model.validate_model(LogisticRegressionCV(), 
                           df[features], df['mid_price_indicator'])
res['features'] = ', '.join(features)
res['method'] = 'logistic'
df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc
0,0.576358,"queue_imbalance, queue_imbalance_2",0.086908,0.087925,logistic,0.544743,0.613639,0.543423,0.580820,0.090957,0.090957,0.549242,0.616475,0.545367
1,0.568687,queue_imbalance,0.084353,0.085012,logistic,0.544689,0.596269,0.542181,0.572303,0.080436,0.080436,0.545268,0.602290,0.540135


In [6]:
gammas = [0.1, 1, 10, 100]
cs = [0.1, 1, 10, 100]
coef0s = [0.1, 1, 10, 100]


for g in gammas:
    for c in cs:
        for coef in coef0s:
            clf = SVC(kernel='sigmoid', gamma=g, C=c, coef0=coef)
            features = ['queue_imbalance', 'prev_queue_imbalance']
            res = model.validate_model(clf, df[features], df['mid_price_indicator'])
            res['features'] = ', '.join(features) 
            res['method'] = 'svm_sigmoid'
            res['gamma'] = g
            res['coef0'] = coef
            res['c'] = c
            df_result = df_result.append(res, ignore_index=True)

            clf = SVC(kernel='sigmoid', gamma=g, C=c, coef0=coef)
            features = ['queue_imbalance']
            res = model.validate_model(clf, df[features], df['mid_price_indicator'])
            res['features'] = ', '.join(features) 
            res['method'] = 'svm_sigmoid'
            res['gamma'] = g
            res['coef0'] = coef
            res['c'] = c
            df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,0.500914,"queue_imbalance, prev_queue_imbalance",0.097412,0.100759,logistic,0.561920,0.460802,0.548728,0.493833,0.092301,0.092301,0.542109,0.456659,0.545859,NaN,NaN,NaN
1,0.488951,queue_imbalance,0.096852,0.101434,logistic,0.564992,0.443385,0.548471,0.479454,0.089059,0.089059,0.542867,0.434906,0.544206,NaN,NaN,NaN
2,0.382354,"queue_imbalance, prev_queue_imbalance",0.067433,0.068830,svm_sigmoid,0.430131,0.378830,0.533746,0.377756,0.063816,0.063816,0.423597,0.369141,0.531826,0.1,0.1,0.1
3,0.381419,queue_imbalance,0.066061,0.067707,svm_sigmoid,0.430325,0.378140,0.533062,0.375889,0.062398,0.062398,0.423157,0.367016,0.531119,0.1,0.1,0.1
4,0.341442,"queue_imbalance, prev_queue_imbalance",0.060885,0.063712,svm_sigmoid,0.372884,0.340204,0.530494,0.331102,0.055691,0.055691,0.363799,0.322945,0.527783,0.1,1.0,0.1
5,0.342108,queue_imbalance,0.061222,0.064340,svm_sigmoid,0.373885,0.340711,0.530668,0.330516,0.055001,0.055001,0.363604,0.321737,0.527436,0.1,1.0,0.1
6,0.000000,"queue_imbalance, prev_queue_imbalance",0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,10.0,0.1
7,0.000000,queue_imbalance,0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,10.0,0.1
8,0.000000,"queue_imbalance, prev_queue_imbalance",0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,100.0,0.1
9,0.000000,queue_imbalance,0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,100.0,0.1


In [7]:
gammas = [0.1, 1, 10, 100]
cs = [0.1, 1, 10, 100]

for g in gammas:
    for c in cs:
        clf = SVC(kernel='rbf', gamma=g, C=c)
        features = ['queue_imbalance', 'prev_queue_imbalance']
        res = model.validate_model(clf, df[features], df['mid_price_indicator'])
        res['features'] = ', '.join(features) 
        res['method'] = 'svm_rbf'
        res['gamma'] = g
        res['c'] = c
        df_result = df_result.append(res, ignore_index=True)

        clf = SVC(kernel='sigmoid', gamma=g, C=c)
        features = ['queue_imbalance']
        res = model.validate_model(clf, df[features], df['mid_price_indicator'])
        res['features'] = ', '.join(features) 
        res['method'] = 'svm_rbf'
        res['gamma'] = g
        res['c'] = c
        df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,0.500914,"queue_imbalance, prev_queue_imbalance",0.097412,0.100759,logistic,0.561920,0.460802,0.548728,0.493833,0.092301,0.092301,0.542109,0.456659,0.545859,NaN,NaN,NaN
1,0.488951,queue_imbalance,0.096852,0.101434,logistic,0.564992,0.443385,0.548471,0.479454,0.089059,0.089059,0.542867,0.434906,0.544206,NaN,NaN,NaN
2,0.382354,"queue_imbalance, prev_queue_imbalance",0.067433,0.068830,svm_sigmoid,0.430131,0.378830,0.533746,0.377756,0.063816,0.063816,0.423597,0.369141,0.531826,0.1,0.1,0.1
3,0.381419,queue_imbalance,0.066061,0.067707,svm_sigmoid,0.430325,0.378140,0.533062,0.375889,0.062398,0.062398,0.423157,0.367016,0.531119,0.1,0.1,0.1
4,0.341442,"queue_imbalance, prev_queue_imbalance",0.060885,0.063712,svm_sigmoid,0.372884,0.340204,0.530494,0.331102,0.055691,0.055691,0.363799,0.322945,0.527783,0.1,1.0,0.1
5,0.342108,queue_imbalance,0.061222,0.064340,svm_sigmoid,0.373885,0.340711,0.530668,0.330516,0.055001,0.055001,0.363604,0.321737,0.527436,0.1,1.0,0.1
6,0.000000,"queue_imbalance, prev_queue_imbalance",0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,10.0,0.1
7,0.000000,queue_imbalance,0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,10.0,0.1
8,0.000000,"queue_imbalance, prev_queue_imbalance",0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,100.0,0.1
9,0.000000,queue_imbalance,0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,100.0,0.1


In [8]:
cs = [0.1, 1, 10, 100]

for c in cs:
    clf = SVC(kernel='linear', C=c)
    features = ['queue_imbalance', 'prev_queue_imbalance']
    res = model.validate_model(clf, df[features], df['mid_price_indicator'])
    res['features'] = ', '.join(features) 
    res['method'] = 'svm_linear'
    res['c'] = c
    df_result = df_result.append(res, ignore_index=True)

    clf = SVC(kernel='sigmoid', gamma=g, C=c)
    features = ['queue_imbalance']
    res = model.validate_model(clf, df[features], df['mid_price_indicator'])
    res['features'] = ', '.join(features) 
    res['method'] = 'svm_linear'
    res['c'] = c
    df_result = df_result.append(res, ignore_index=True)
df_result

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma
0,0.500914,"queue_imbalance, prev_queue_imbalance",0.097412,0.100759,logistic,0.561920,0.460802,0.548728,0.493833,0.092301,0.092301,0.542109,0.456659,0.545859,NaN,NaN,NaN
1,0.488951,queue_imbalance,0.096852,0.101434,logistic,0.564992,0.443385,0.548471,0.479454,0.089059,0.089059,0.542867,0.434906,0.544206,NaN,NaN,NaN
2,0.382354,"queue_imbalance, prev_queue_imbalance",0.067433,0.068830,svm_sigmoid,0.430131,0.378830,0.533746,0.377756,0.063816,0.063816,0.423597,0.369141,0.531826,0.1,0.1,0.1
3,0.381419,queue_imbalance,0.066061,0.067707,svm_sigmoid,0.430325,0.378140,0.533062,0.375889,0.062398,0.062398,0.423157,0.367016,0.531119,0.1,0.1,0.1
4,0.341442,"queue_imbalance, prev_queue_imbalance",0.060885,0.063712,svm_sigmoid,0.372884,0.340204,0.530494,0.331102,0.055691,0.055691,0.363799,0.322945,0.527783,0.1,1.0,0.1
5,0.342108,queue_imbalance,0.061222,0.064340,svm_sigmoid,0.373885,0.340711,0.530668,0.330516,0.055001,0.055001,0.363604,0.321737,0.527436,0.1,1.0,0.1
6,0.000000,"queue_imbalance, prev_queue_imbalance",0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,10.0,0.1
7,0.000000,queue_imbalance,0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,10.0,0.1
8,0.000000,"queue_imbalance, prev_queue_imbalance",0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,100.0,0.1
9,0.000000,queue_imbalance,0.000000,0.000000,svm_sigmoid,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.1,100.0,0.1


In [9]:
df_result['stock'] = [stock] * len(df_result)
df_result.to_csv('res_{}_queue_imbalance_second_feature.csv'.format(stock))

In [10]:
df_result.sort_values(by='f1', ascending=False).head(5)

,f1,features,kappa,matthews,method,precision,recall,roc_auc,train_f1,train_kappa,train_matthews,train_precision,train_recall,train_roc_auc,c,coef0,gamma,stock
150,0.512400,"queue_imbalance, prev_queue_imbalance",0.074970,0.075762,svm_rbf,0.541483,0.490670,0.537460,0.543320,0.144491,0.144491,0.568274,0.522564,0.571893,10.0,NaN,10.0,9061
169,0.511947,queue_imbalance,0.030397,0.030539,svm_linear,0.515179,0.509897,0.515209,0.497909,0.025028,0.025028,0.498005,0.497813,0.512514,100.0,NaN,NaN,9061
161,0.511947,queue_imbalance,0.030397,0.030539,svm_rbf,0.515179,0.509897,0.515209,0.497909,0.025028,0.025028,0.498005,0.497813,0.512514,100.0,NaN,100.0,9061
123,0.511947,queue_imbalance,0.030397,0.030539,svm_sigmoid,0.515179,0.509897,0.515209,0.497909,0.025028,0.025028,0.498005,0.497813,0.512514,100.0,0.1,100.0,9061
101,0.511865,queue_imbalance,0.030153,0.030239,svm_sigmoid,0.514750,0.510028,0.515089,0.497349,0.024390,0.024390,0.497518,0.497205,0.512202,0.1,1.0,100.0,9061
